In [7]:
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics 

In [8]:
import multiprocessing

In [9]:
import mkl
import time
mkl.set_num_threads(1)

1

In [10]:
mkl.get_max_threads()

1

In [11]:
from conf import *
from numba import njit, jit, jitclass, int8, complex128, int64

In [6]:
h = 1.0
V = 1.0

length = [8, 8]
name = 'h={}V={}l={}'.format(h, V, length)
kernel = 1
n_chains = 1
n_discards = 1
n_samples = 1e5

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)

4


In [573]:
spec = [
    ("local_states", int8[:]),
    ("size", int64), 
    ("n_states", int64),
    ("basis", int64[:]),
    ("constant", int64),
    ("diag_mels", complex128[:,:]),
    ("n_conns", int64[:,:]),
    ("mels", complex128[:,:,:]),
    ("x_prime", int8[:,:,:]),
    ("acting_on", int64[:,:]),
    ("acting_size", int64[:]),
#     ("acting_size", int64),
    
]

In [600]:
get_conn1 = nk.operator._local_operator.LocalOperator._get_conn_flattened_kernel
get_conn2 = nk.operator._local_operator.DimerLocalOperator2._get_conn_flattened_kernel
import numpy as _np

@jitclass(spec)
class _DimerLocalKernel_1:
    def __init__(self, 
                local_states, 
                size,
                basis,
                constant,
                diag_mels,
                n_conns,
                mels,
                x_prime,
                acting_on,
                acting_size,
                ):

        self.local_states = _np.sort(local_states)
        self.size = size
        self.n_states = self.local_states.size
        self.basis = basis[::-1].copy()
#         self.basis = basis
        self.constant = constant
        self.diag_mels = diag_mels
        self.n_conns = n_conns
        self.mels = mels
        self.x_prime = x_prime
        self.acting_on = acting_on
#         self.acting_size = int(acting_size[0])
        self.acting_size = acting_size

        
    
    def transition(self, state, state_1, w, r, sweep_size):
        '''
        This transition is exclusively for batch_size = 1
        '''

        accepted = 0
        batch_size = state.shape[0]
        
        assert batch_size == 1, 'batch_size must be 1'
        
        sections = _np.zeros(1, dtype=_np.int64)
        sections_1 = _np.zeros(1, dtype=_np.int64)
        state_1 = state

        log_values = _log_val_kernel(state.astype(_np.float64), w, r)[0]
#         print(log_values)
        
        state_prime = self.get_conn(state,sections)
        n_conn = sections[0]-1
#         print(log_values_prime)
        

        while True:
            
#             state_prime_, mels_ = self.get_conn(state, sections)
#             n_conn_ = sections[0]-1

            # print((state_prime_ == state_prime).any())
            # print((n_conn_ + 1).sum())
#             print(log_values)
            
            rs = (_np.random.rand(1) * (n_conn)).astype(_np.int64)

            state_1 = state_prime[rs+1].reshape(1,-1)

            state_1_prime = self.get_conn(state_1, sections_1)
            n_conn_1 = sections_1[0]-1

            prob_corr = n_conn/n_conn_1
            
            
            log_values_1 = _log_val_kernel(state_1.astype(_np.float64), w, r)[0]

            prob = _np.exp(
                2 * (log_values_1 - log_values)
            ) * prob_corr
            
            if prob > _np.random.rand(1):
                state[:] = state_1
                state_prime = state_1_prime
                log_values = log_values_1
                n_conn = n_conn_1
                accepted += 1
                

            # sections = sections_1

            
            if accepted >= sweep_size:
                break
                
        return accepted


    def random_state(self, state):

        for i in range(state.shape[0]):
#         for si in range(state.shape[1]):
            rs = _random.randint(0, self.n_states)
            state[i] = _np.zeros_like(state[i]) + self.local_states[rs]
    






    def get_conn(self, x, sections):

        return get_conn_one(
            x,
            sections,
            self.basis,
            self.n_conns,
            self.x_prime,
            self.acting_on,
        )

#         return get_conn_1(
#             x,
#             sections,
#             self.basis,
#             self.constant,
#             self.diag_mels,
#             self.n_conns,
#             self.mels,
#             self.x_prime,
#             self.acting_on,
#             self.acting_size,
#         )

@jit(fastmath=True)
def _log_cosh_sum(x, out, add_factor=None):
    x = x * _np.sign(x.real)
    if add_factor is None:
        for i in range(x.shape[0]):
            out[i] = _np.sum(x[i] - _np.log(2.0) + _np.log(1.0 + _np.exp(-2.0 * x[i])))
    else:
        for i in range(x.shape[0]):
            out[i] += add_factor * (
                _np.sum(x[i] - _np.log(2.0) + _np.log(1.0 + _np.exp(-2.0 * x[i])))
            )
    return out



@jit(nopython=True)
def _log_val_kernel(x, W, r):

    if x.ndim != 2:
        raise RuntimeError("Invalid input shape, expected a 2d array")

    # if out is None:
    out = _np.empty(x.shape[0], dtype=_np.float64)
    r = x.dot(W)
    _log_cosh_sum(r, out)

    return out


<ipython-input-600-1f5a658c619f>:5: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [601]:
@jit(nopython=True)
def get_conn_one(
    x,
    sections,
    basis,
    n_conns,
    all_x_prime,
    acting_on,
):
    batch_size = x.shape[0]
    n_sites = x.shape[1]

    assert batch_size == 1, 'batch_size must be 1'

    n_operators = n_conns.shape[0]

    
    X_temp = _np.zeros((n_operators, n_sites), dtype=x.dtype)



#     X_temp[0] = x
    S = 1
    for b in range(batch_size):
        x_b = x[b]
        # diagonal element
        # counting the off-diagonal elements
        for i in range(n_operators):
            a = 15
            
            acting_on_i = acting_on[i]
            x_i = x_b[acting_on_i]

            for k in range(4):
                a += (
                    x_i[k]
                    * basis[k]
                )
            a = int(a/2)
            
            if n_conns[i, a]:
                X_temp[S] = x_b
                X_temp[S][acting_on_i] = all_x_prime[i, a]
                S += 1
                

#         tot_conn += conn_b
        sections[b] = S
    



    return X_temp[:S]

In [586]:
x = np.random.randint(0,2,10)

In [587]:
op._n_conns[0]

array([0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0])

In [588]:
x = np.ones((1,128), dtype=np.int8)
y = np.copy(x)


In [597]:
%timeit x

13 ns ± 0.489 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [589]:
_x_prime = op_transition._x_prime[:,:,0,:]
transition = _DimerLocalKernel_1(
    op_transition._local_states,
    op_transition._size,
    op_transition._basis,
    op_transition._constant,
    op_transition._diag_mels,
    op_transition._n_conns,
    op_transition._mels,
    _x_prime,
    op_transition._acting_on,
    op_transition._acting_size


)

In [590]:
_x_prime.shape

(384, 16, 4)

In [591]:
s = np.zeros(1, dtype=np.int)


In [592]:
%timeit P = transition.get_conn(x, s)

15.2 µs ± 8.29 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [605]:
%timeit transition.transition(x,y, ma._w, ma._r,400)

5.94 ms ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [160]:
P = transition.get_conn(x, np.ones(1))

In [524]:
%timeit P1 = op_transition.get_conn(x)[0]

23.3 µs ± 523 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [310]:
P1[1] == P[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]

In [184]:
np.where(op_transition._x_prime[0])

(array([ 4,  4,  4,  4,  7,  7,  7,  7,  8,  8,  8,  8, 11, 11, 11, 11]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]))

In [185]:
op_transition._x_prime[0][4]

array([[-1, -1,  1, -1],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0],
       [ 0,  0,  0,  0]], dtype=int8)

In [197]:
state = []
num = 4
for i in range(4):
    state.append(num // 2 ** (3-i))
    num %=  2 ** (3-i)
state = np.array(state)*2 -1

In [198]:
state

array([-1,  1, -1, -1])